In [230]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/System-Threat-Forecaster/sample_submission.csv
/kaggle/input/System-Threat-Forecaster/train.csv
/kaggle/input/System-Threat-Forecaster/test.csv


In [231]:
def sumbit_to_competition(model):
    test_set = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")
    pred = model.predict(test_set)

    submission = pd.DataFrame({"id": range(0,test_set.shape[0]), "target": pred}) 

    submission.to_csv('submission.csv',index=False)
    print(submission.head(5)) 

# EDA

In [95]:
train_df = pd.read_csv('/kaggle/input/System-Threat-Forecaster/train.csv')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 76 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   MachineID                           100000 non-null  object 
 1   ProductName                         100000 non-null  object 
 2   EngineVersion                       100000 non-null  object 
 3   AppVersion                          100000 non-null  object 
 4   SignatureVersion                    100000 non-null  object 
 5   IsBetaUser                          100000 non-null  int64  
 6   RealTimeProtectionState             99934 non-null   float64
 7   IsPassiveModeEnabled                100000 non-null  int64  
 8   AntivirusConfigID                   99924 non-null   float64
 9   NumAntivirusProductsInstalled       99924 non-null   float64
 10  NumAntivirusProductsEnabled         99924 non-null   float64
 11  HasTpm                     

In [96]:
test_df=pd.read_csv('/kaggle/input/System-Threat-Forecaster/test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 75 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   MachineID                           10000 non-null  object 
 1   ProductName                         10000 non-null  object 
 2   EngineVersion                       10000 non-null  object 
 3   AppVersion                          10000 non-null  object 
 4   SignatureVersion                    10000 non-null  object 
 5   IsBetaUser                          10000 non-null  int64  
 6   RealTimeProtectionState             9991 non-null   float64
 7   IsPassiveModeEnabled                10000 non-null  int64  
 8   AntivirusConfigID                   9998 non-null   float64
 9   NumAntivirusProductsInstalled       9998 non-null   float64
 10  NumAntivirusProductsEnabled         9998 non-null   float64
 11  HasTpm                              10000 

In [97]:
print(train_df.shape)
print(test_df.shape)

(100000, 76)
(10000, 75)


In [98]:
train_df.isna().sum()

MachineID             0
ProductName           0
EngineVersion         0
AppVersion            0
SignatureVersion      0
                   ... 
IsGamer             559
RegionIdentifier    559
DateAS                0
DateOS               24
target                0
Length: 76, dtype: int64

In [99]:
train_df.nunique()

MachineID           99835
ProductName             2
EngineVersion          37
AppVersion             69
SignatureVersion     2735
                    ...  
IsGamer                 2
RegionIdentifier       15
DateAS               2695
DateOS                112
target                  2
Length: 76, dtype: int64

### Numerical Data

In [100]:
numerical_df = train_df.select_dtypes(include=['int64','float64'])
numerical_df.head(20)

,IsBetaUser,RealTimeProtectionState,IsPassiveModeEnabled,AntivirusConfigID,NumAntivirusProductsInstalled,NumAntivirusProductsEnabled,HasTpm,CountryID,CityID,GeoRegionID,...,FirmwareManufacturerID,FirmwareVersionID,IsSecureBootEnabled,IsVirtualDevice,IsTouchEnabled,IsPenCapable,IsAlwaysOnAlwaysConnectedCapable,IsGamer,RegionIdentifier,target
0,0,7.0,0,53447.0,1.0,1.0,1,51,120232.0,98.0,...,513.0,21964.0,0,0.0,1,0,1.0,0.0,6.0,0
1,0,7.0,0,53447.0,1.0,1.0,1,141,112854.0,167.0,...,628.0,44548.0,1,0.0,0,0,0.0,0.0,10.0,1
2,0,7.0,0,53447.0,1.0,1.0,1,51,41759.0,98.0,...,142.0,9414.0,0,0.0,0,0,0.0,1.0,6.0,1
3,0,7.0,0,46413.0,2.0,1.0,1,68,19507.0,276.0,...,554.0,33060.0,1,0.0,0,0,0.0,0.0,12.0,1
4,0,7.0,0,40466.0,2.0,1.0,1,43,117801.0,53.0,...,628.0,13224.0,0,0.0,0,0,0.0,1.0,7.0,0
5,0,7.0,0,53447.0,1.0,1.0,1,35,36157.0,45.0,...,355.0,20050.0,0,0.0,0,0,0.0,1.0,4.0,0
6,0,7.0,0,53447.0,1.0,1.0,1,91,145233.0,277.0,...,142.0,3561.0,0,0.0,0,0,0.0,0.0,11.0,0
7,0,7.0,0,53447.0,1.0,1.0,1,18,37376.0,277.0,...,554.0,33115.0,0,0.0,0,0,0.0,0.0,1.0,1
8,0,7.0,0,53447.0,1.0,1.0,1,18,37378.0,277.0,...,127.0,63086.0,0,0.0,0,0,0.0,0.0,1.0,0
9,0,7.0,0,53447.0,1.0,1.0,1,158,117594.0,202.0,...,554.0,33111.0,1,0.0,0,0,0.0,1.0,1.0,1


In [168]:
for i in numerical_df:
    print(numerical_df[i].unique().sum())

0
nan
1
nan
nan
nan
1
24399
nan
34853.0
23372
384654
2801
nan
0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1
nan
nan
nan
nan
nan
381926
777444
nan
5021
1
nan
nan
nan
1
nan
1
1
nan
nan
nan
1


### Categorical Data

In [102]:
categorical_df = train_df.select_dtypes(include=['object'])
categorical_df.head()

,MachineID,ProductName,EngineVersion,AppVersion,SignatureVersion,PlatformType,Processor,OSVersion,OsPlatformSubRelease,OSBuildLab,...,OSBranch,OSEdition,OSSkuFriendlyName,OSInstallType,AutoUpdateOptionsName,OSGenuineState,LicenseActivationChannel,FlightRing,DateAS,DateOS
0,f541bae429089117c4aac39c90dd3416,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1003.0,windows10,x86,10.0.0.0,rs1,14393.2214.x86fre.rs1_release_1.180402-1758,...,rs1_release,Core,CORE,Update,UNKNOWN,IS_GENUINE,Retail,Retail,2018-09-10 10:11:00,2018-04-17
1,dc2b14d9ce3a0ce4050bb640190f2ca5,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1465.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,...,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,FullAuto,IS_GENUINE,OEM:DM,Retail,2018-08-16 00:01:00,2018-08-14
2,fd20c5f010e9c5f91ad1c6b3e0da68a0,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1546.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,...,rs4_release,Core,CORE,Reset,FullAuto,IS_GENUINE,OEM:NONSLP,Retail,2018-09-20 23:20:00,2018-09-11
3,38711eae85eb77a72ec5dfdf27eb2a76,win8defender,1.1.15200.1,4.12.17007.18011,1.275.1141.0,windows10,x64,10.0.0.0,rs2,15063.0.amd64fre.rs2_release.170317-1834,...,rs2_release,Core,CORE,Upgrade,UNKNOWN,IS_GENUINE,OEM:DM,Retail,2018-09-14 00:32:00,2018-01-03
4,32607c9a543a9214e2c7e45800ed4849,win8defender,1.1.15200.1,4.13.17134.228,1.275.1283.0,windows10,x86,10.0.0.0,rs4,17134.1.x86fre.rs4_release.180410-1804,...,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,FullAuto,IS_GENUINE,Retail,Retail,2018-09-15 19:34:00,2018-09-11


# Preprocessing

In [232]:
from sklearn.model_selection import train_test_split

X = train_df.drop('target', axis=1)
y = train_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(train_df.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(100000, 76)
(80000, 75) (80000,)
(20000, 75) (20000,)


In [233]:
num_cols = X_train.select_dtypes(include=['int64','float64']).columns
num_cols

Index(['IsBetaUser', 'RealTimeProtectionState', 'IsPassiveModeEnabled',
       'AntivirusConfigID', 'NumAntivirusProductsInstalled',
       'NumAntivirusProductsEnabled', 'HasTpm', 'CountryID', 'CityID',
       'GeoRegionID', 'LocaleEnglishNameID', 'OSBuildNumber', 'OSProductSuite',
       'IsSystemProtected', 'AutoSampleSubmissionEnabled', 'SMode',
       'IEVersionID', 'FirewallEnabled', 'EnableLUA', 'OEMNameID',
       'OEMModelID', 'ProcessorCoreCount', 'ProcessorManufacturerID',
       'ProcessorModelID', 'PrimaryDiskCapacityMB', 'SystemVolumeCapacityMB',
       'HasOpticalDiskDrive', 'TotalPhysicalRAMMB',
       'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
       'PrimaryDisplayResolutionVertical', 'InternalBatteryNumberOfCharges',
       'OSBuildNumberOnly', 'OSBuildRevisionOnly', 'OSInstallLanguageID',
       'OSUILocaleID', 'IsPortableOS', 'IsFlightsDisabled',
       'FirmwareManufacturerID', 'FirmwareVersionID', 'IsSecureBootEnabled',
       'IsVirtua

In [234]:
cat_cols = X_train.select_dtypes(include="object").columns
cat_cols

Index(['MachineID', 'ProductName', 'EngineVersion', 'AppVersion',
       'SignatureVersion', 'PlatformType', 'Processor', 'OSVersion',
       'OsPlatformSubRelease', 'OSBuildLab', 'SKUEditionName',
       'MDC2FormFactor', 'DeviceFamily', 'PrimaryDiskType', 'ChassisType',
       'PowerPlatformRole', 'NumericOSVersion', 'OSArchitecture', 'OSBranch',
       'OSEdition', 'OSSkuFriendlyName', 'OSInstallType',
       'AutoUpdateOptionsName', 'OSGenuineState', 'LicenseActivationChannel',
       'FlightRing', 'DateAS', 'DateOS'],
      dtype='object')

In [235]:
threshold = 5

oh_cols = [i for i in cat_cols if len(X_train[i].unique()) <= threshold]
oh_cols

['ProductName',
 'PlatformType',
 'Processor',
 'DeviceFamily',
 'PrimaryDiskType',
 'OSArchitecture',
 'OSGenuineState']

In [236]:
ord_cols = [i for i in cat_cols if i not in oh_cols]
ord_cols

['MachineID',
 'EngineVersion',
 'AppVersion',
 'SignatureVersion',
 'OSVersion',
 'OsPlatformSubRelease',
 'OSBuildLab',
 'SKUEditionName',
 'MDC2FormFactor',
 'ChassisType',
 'PowerPlatformRole',
 'NumericOSVersion',
 'OSBranch',
 'OSEdition',
 'OSSkuFriendlyName',
 'OSInstallType',
 'AutoUpdateOptionsName',
 'LicenseActivationChannel',
 'FlightRing',
 'DateAS',
 'DateOS']

In [237]:
date_cols= ['DateAS','DateOS']
ord_cols= [x for x in ord_cols if x not in date_cols]

print(ord_cols)
print(date_cols)

['MachineID', 'EngineVersion', 'AppVersion', 'SignatureVersion', 'OSVersion', 'OsPlatformSubRelease', 'OSBuildLab', 'SKUEditionName', 'MDC2FormFactor', 'ChassisType', 'PowerPlatformRole', 'NumericOSVersion', 'OSBranch', 'OSEdition', 'OSSkuFriendlyName', 'OSInstallType', 'AutoUpdateOptionsName', 'LicenseActivationChannel', 'FlightRing']
['DateAS', 'DateOS']


In [239]:
from sklearn.base import BaseEstimator, TransformerMixin

class DateTimeFeaturesTransformer(BaseEstimator, TransformerMixin):
  """
  Custom transformer for extracting integer features (year, month, day, hour, minute, second)
  from datetime strings in the format 'DD-MM-YYYY HH:MM:SS'. Handles both DataFrame and Series inputs.
  """

  def __init__(self, date_format='%Y-%m-%d %H:%M:%S'):
    """
    Args:
      date_format: The format string used to parse the datetime strings. Defaults to '%d-%m-%Y %H:%M:%S'.
    """
    self.date_format = date_format

  def fit(self, X, y=None):
    """
    This method is required for scikit-learn pipelines but does nothing in this case.
    """
    return self

  def transform(self, X):
    """
    Transforms the input data by extracting integer features from datetime strings.

    Args:
      X: The input data, can be a pandas DataFrame or Series.

    Returns:
      A NumPy array containing the extracted integer features (year, month, day, hour, minute, second).
    """

    # Convert to pandas Series if DataFrame is passed
    if isinstance(X, pd.DataFrame):
      X = X.iloc[:, 0]

    # Convert to datetime
    datetimes = pd.to_datetime(X, format=self.date_format)

    # Extract features
    year = datetimes.dt.year.astype(int)
    month = datetimes.dt.month.astype(int)
    day = datetimes.dt.day.astype(int)
    hour = datetimes.dt.hour.astype(int)
    minute = datetimes.dt.minute.astype(int)
    second = datetimes.dt.second.astype(int)

    # Combine features into a NumPy array
    features = np.column_stack([year, month, day, hour, minute, second])

    return features

### Pipeline

In [240]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer

preprocessing_pipe = ColumnTransformer(
    transformers=[
        
        # Pipeline for ordinal columns: imputing most frequent and ordinal encoding
        ('ord_pipeline', Pipeline([
            ('impute_most_frequent', SimpleImputer(strategy='most_frequent')),
            ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ]), ord_cols),
        
        # Pipeline for 'onehot columns: imputing most frequent and one-hot encoding
        ('oh_pipeline', Pipeline([
            ('impute_most_frequent', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'))
        ]), oh_cols),

        # Pipeline for numerical columns: imputing most frequent and one-hot encoding
        ('num_pipeline', Pipeline([
            ('impute_most_frequent', SimpleImputer(strategy='most_frequent')),
            ('stdScl', StandardScaler())
        ]), num_cols),
        
        # Handle date
        ('date_transformer', DateTimeFeaturesTransformer(), date_cols)
    ]
)

In [180]:
X_train_pre = preprocessing_pipe.fit_transform(X_train)
len(X_train_pre[0])

94

In [175]:
#X_train_pre = pd.DataFrame(X_train_pre, columns=X_train_pre[0])

# Modelling

In [241]:
#imports

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

## Model 0: Dummy Classifier

In [113]:
model = DummyClassifier().fit(X,y)
y_pred = model.predict(X)

In [114]:
from sklearn.metrics import accuracy_score

accuracy_score(y,y_pred)

0.50525

## Model 1: Logistic Regression

In [212]:
#model_pipe_lr = Pipeline([
#    ('preprocess', preprocessing_pipe),
#    ('lr_model', LogisticRegression(random_state=42, solver='liblinear', max_iter=2000))
#])

In [185]:
#lr_model = model_pipe_lr.fit(X_train, y_train)
#lr_y_pred = lr_model.predict(X_test)

In [186]:
#accuracy_score(y_test,lr_y_pred)
# 0.59475

0.59475

In [190]:
#sumbit_to_competition(lr_model)

   id  target
0   0       1
1   1       0
2   2       1
3   3       1
4   4       0


#### Hyperparameter Tuning

In [201]:
#model_pipe_logistic = Pipeline([
#    ('preprocess', preprocessing_pipe),
#    ('logistic_model', LogisticRegressionCV(
#        Cs=[0.01, 0.1, 1, 10, 100],
#        cv=5,                       
#        penalty='l2',
#        scoring='f1_macro',
#        max_iter=2000,
 #       random_state=42 
#    ))
#])

In [204]:
#lr_cv_model = model_pipe_logistic.fit(X_train, y_train)
#lr_cv_y_pred = lr_cv_model.predict(X_test)

In [205]:
#accuracy_score(y_test,lr_cv_y_pred)
# 0.57345

0.57345

## Model 2: KNN

In [191]:
#knn_model_pipe = Pipeline([
#    ('preprocess', preprocessing_pipe),
#    ('knn_model', KNeighborsClassifier())
#])

In [192]:
#knn_model = knn_model_pipe.fit(X_train, y_train)
#knn_y_pred = knn_model.predict(X_test)

In [193]:
#accuracy_score(y_test,knn_y_pred)
# 0.5165

0.5165

#### Hyperparameter Tuning

In [209]:
#gridcv_knn_model_pipe = Pipeline([
#    ('preprocess', preprocessing_pipe),
#    ('knn_model', KNeighborsClassifier(n_neighbors=3))
#])

In [210]:
#gridcv_knn_model = gridcv_knn_model_pipe.fit(X_train, y_train)
#gridcv_knn_y_pred = gridcv_knn_model.predict(X_test)

In [211]:
#accuracy_score(y_test,gridcv_knn_y_pred)
# 0.5146

0.5146

## Model 3: RandomForestClassifier

In [214]:
#rf_model_pipe = Pipeline([
#    ('preprocess', preprocessing_pipe),
#    ('rf_model', RandomForestClassifier())
#])

In [215]:
#rf_model = rf_model_pipe.fit(X_train, y_train)
#rf_y_pred = rf_model.predict(X_test)

In [216]:
#accuracy_score(y_test,rf_y_pred)

0.6107

In [217]:
#sumbit_to_competition(rf_model)

   id  target
0   0       1
1   1       1
2   2       0
3   3       0
4   4       0


#### Hyperparameter Tuning (RandomizedSearchCV)

took way too much time (didn't execute)

In [ ]:
#randomcv_model_pipe = Pipeline([
 #   ('preprocess', preprocessing_pipe),
  #  ('rf_model_randomcv', RandomForestClassifier(
   #                                     max_depth=20, 
    #                                    max_features=None,
     #                                   min_samples_leaf=2,
      #                                  n_estimators=300,
       #                                 class_weight='balanced')
    #)
#])
#randomcv_rf_model = randomcv_model_pipe.fit(X_train, y_train)
#randomcv_rf_y_pred = randomcv_rf_model.predict(X_test)

## Model 4: HistGradientBoostingClassifier

In [242]:
hgbc_model_pipe = Pipeline([
    ('preprocess', preprocessing_pipe),
    ('hgbc_model', HistGradientBoostingClassifier(class_weight='balanced'))
])

In [243]:
hgbc_model = hgbc_model_pipe.fit(X_train, y_train)
hgbc_y_pred = hgbc_model.predict(X_test)

In [244]:
accuracy_score(y_test,hgbc_y_pred)

0.62235

In [245]:
sumbit_to_competition(hgbc_model)

   id  target
0   0       1
1   1       0
2   2       1
3   3       0
4   4       0
